<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Импорт-библиотек,-загрузка-датасета" data-toc-modified-id="Импорт-библиотек,-загрузка-датасета-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Импорт библиотек, загрузка датасета</a></span></li><li><span><a href="#Создание-датафрейма-с-репликами-менеджеров,-пустого-датафрейма-для-результатов-парсинга" data-toc-modified-id="Создание-датафрейма-с-репликами-менеджеров,-пустого-датафрейма-для-результатов-парсинга-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Создание датафрейма с репликами менеджеров, пустого датафрейма для результатов парсинга</a></span></li><li><span><a href="#Парсинг-диалогов,-с-заполнением-датафрейма-результатами-парсинга" data-toc-modified-id="Парсинг-диалогов,-с-заполнением-датафрейма-результатами-парсинга-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Парсинг диалогов, с заполнением датафрейма результатами парсинга</a></span></li><li><span><a href="#Запись-результатов-парсинга-в-файл" data-toc-modified-id="Запись-результатов-парсинга-в-файл-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Запись результатов парсинга в файл</a></span></li></ul></div>

## Импорт библиотек, загрузка датасета

In [1]:
import pandas as pd
# !pip install nltk
import nltk
# !pip install pymorphy2
import pymorphy2
from itertools import takewhile

In [2]:
df = pd.read_csv('test_data.csv')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


## Создание датафрейма с репликами менеджеров, пустого датафрейма для результатов парсинга

In [3]:
# Создание датафрейма с репликами менеджеров
df_mng = df.query('role =="manager"')
df_mng.head()

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет


In [4]:
# Создание датафрейма для заполнения результатов парсинга
df_pars = pd.DataFrame({'dlg_id': pd.Series(dtype='int'),
                   'greeting': pd.Series(dtype='str'),
                   'int_himself': pd.Series(dtype='str'),
                   'name': pd.Series(dtype='str'),
                   'company': pd.Series(dtype='str'),
                   'farewell': pd.Series(dtype='str'),
                   'chek': pd.Series(dtype='bool')})
df_pars['dlg_id'] = df_mng.dlg_id.unique()
display(df_pars)

,dlg_id,greeting,int_himself,name,company,farewell,chek
0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN
5,5,NaN,NaN,NaN,NaN,NaN,NaN


## Парсинг диалогов, с заполнением датафрейма результатами парсинга

In [5]:
greeting = {'здравствуйте', 'добрый', 'привет'}
farewell = {'хороший', 'хорошего', 'свидание',
            'добрый', 'доброго'}
himself = {'звать', 'это'}

morph = pymorphy2.MorphAnalyzer()
for index, dlg in enumerate(df_pars['dlg_id']):
    txt_grt = '' # Реплики с приветствиями
    company = [] # Название компаний
    
    for txt in df_mng.query('dlg_id == @dlg').iloc[:3, 3]: #  Выбор первых 3-х реплик в диалоге
        token = nltk.word_tokenize(txt)
        normal_form = []
        name = 'no_name'
              
        for word in token:
            p = morph.parse(word)[0]
            normal_form.append(p.normal_form)
            
            # находим имя человека и присваиваем его переменной 
            if 'Name' in p.tag and p.score >= 0.4:
                name = p.normal_form
                
        # Заполнение столбца с имененами менеджеров и
        # заполнение столбца с репликами, где менеджер представился
        if himself.intersection(normal_form) and name != 'no_name':
            df_pars.loc[index, 'name'] = name
            df_pars.loc[index, 'int_himself'] = txt        

        # Заполнение столбца с названиями компаний    
        if 'компания' in normal_form:           
            index_begin = normal_form.index('компания') + 1
            company = takewhile(lambda x: morph.parse(x)[0].tag.POS in ['NOUN', 'ADJF'], token[index_begin:])
            df_pars.loc[index, 'company'] = ' '.join(list(company))
          
        # Заполнение переменной репликами, где менеджер поздоровался
        # таких реплик может быть несколько, каждая реплика - предложение
        if greeting.intersection(normal_form):
            txt_grt = txt_grt + '. ' + txt
            
    # Заполнение столбца с репликами, где менеджер поздоровался
    if txt_grt[2:] != '':
        df_pars.loc[index, 'greeting'] = txt_grt[2:]

    #  Выбор последних 3-х реплик менеджера в диалоге 
    for txt in df_mng.query('dlg_id == @dlg').iloc[-3:, 3]:
        txt_frw = ''
        token = nltk.word_tokenize(txt)
        normal_form = []
        for word in token:
            p = morph.parse(word)[0]
            normal_form.append(p.normal_form)
            
        # Заполнение переменной репликами, где менеджер попрощался
        # таких реплик может быть несколько, каждая реплика - предложение
        if farewell.intersection(normal_form):
            txt_frw = txt_frw + '. ' + txt
    
    # Заполнение столбца с репликами, где менеджер попрощался
    if txt_frw[2:] != '':
        df_pars.loc[index, 'farewell'] = txt_frw[2:]
        
# Заполнение столбца проверки требования "поздороваться и попрощаться с клиентом"               
df_pars['chek'] =  df_pars['greeting'].notna() & df_pars['farewell'].notna()

# Вывод датафрейма с результатами парсинга
pd.set_option('display.max_colwidth', None)
display(df_pars)

,dlg_id,greeting,int_himself,name,company,farewell,chek
0,0,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина,диджитал бизнес,Всего хорошего до свидания,True
1,1,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,ангелина,диджитал бизнес,До свидания,True
2,2,Алло здравствуйте,Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,ангелина,диджитал бизнес,NaN,False
3,3,Алло дмитрий добрый день. Добрый меня максим зовут компания китобизнес удобно говорить,Добрый меня максим зовут компания китобизнес удобно говорить,максим,китобизнес,Угу все хорошо да понедельника тогда всего доброго,True
4,4,NaN,NaN,NaN,NaN,Во вторник все ну с вами да тогда до вторника до свидания,False
5,5,NaN,Да это анастасия,анастасия,NaN,Ну до свидания хорошего вечера,False


## Запись результатов парсинга в файл

In [6]:
df_pars.to_csv('pars_result.csv', encoding='utf-8', index=False)